In [1]:
from datasets import load_dataset

imdb = load_dataset("imdb")

In [2]:
imdb["test"][0]


{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/electra-large-discriminator")

In [4]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True) #, max_length=1024)

In [5]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [6]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
import evaluate

accuracy = evaluate.load("accuracy")

In [8]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [9]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [10]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "google/electra-large-discriminator", num_labels=2, id2label=id2label, label2id=label2id
)

# for param in model.transformer.parameters():
#     param.requires_grad = False


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="my_awesome_electra_large",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    warmup_steps=100,
    # fp16=True,
    # load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\Abstract\mambaforge\envs\sentenv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1562 [00:00<?, ?it/s]

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.4989, 'learning_rate': 2e-05, 'epoch': 0.13}
{'loss': 0.2297, 'learning_rate': 1.863201094391245e-05, 'epoch': 0.26}
{'loss': 0.1856, 'learning_rate': 1.72640218878249e-05, 'epoch': 0.38}
{'loss': 0.1662, 'learning_rate': 1.5896032831737347e-05, 'epoch': 0.51}
{'loss': 0.1637, 'learning_rate': 1.4528043775649796e-05, 'epoch': 0.64}
{'loss': 0.1547, 'learning_rate': 1.3160054719562245e-05, 'epoch': 0.77}
{'loss': 0.1467, 'learning_rate': 1.1792065663474691e-05, 'epoch': 0.9}


  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.13312098383903503, 'eval_accuracy': 0.95992, 'eval_runtime': 743.435, 'eval_samples_per_second': 33.628, 'eval_steps_per_second': 4.203, 'epoch': 1.0}
{'loss': 0.1239, 'learning_rate': 1.0424076607387142e-05, 'epoch': 1.02}
{'loss': 0.0983, 'learning_rate': 9.05608755129959e-06, 'epoch': 1.15}
{'loss': 0.1026, 'learning_rate': 7.688098495212039e-06, 'epoch': 1.28}
{'loss': 0.0857, 'learning_rate': 6.3201094391244875e-06, 'epoch': 1.41}
{'loss': 0.0944, 'learning_rate': 4.952120383036936e-06, 'epoch': 1.54}
{'loss': 0.0844, 'learning_rate': 3.584131326949385e-06, 'epoch': 1.66}
{'loss': 0.1111, 'learning_rate': 2.2161422708618332e-06, 'epoch': 1.79}
{'loss': 0.086, 'learning_rate': 8.481532147742819e-07, 'epoch': 1.92}


  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.11906545609235764, 'eval_accuracy': 0.96464, 'eval_runtime': 748.574, 'eval_samples_per_second': 33.397, 'eval_steps_per_second': 4.175, 'epoch': 2.0}
{'train_runtime': 5645.7376, 'train_samples_per_second': 8.856, 'train_steps_per_second': 0.277, 'train_loss': 0.15286866925590017, 'epoch': 2.0}


TrainOutput(global_step=1562, training_loss=0.15286866925590017, metrics={'train_runtime': 5645.7376, 'train_samples_per_second': 8.856, 'train_steps_per_second': 0.277, 'train_loss': 0.15286866925590017, 'epoch': 2.0})